In [ ]:
import io
from datetime import timedelta
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from qiskit import QuantumCircuit
from qiskit.circuit.library import (
    PauliFeatureMap,
    RealAmplitudes,
    ZFeatureMap,
    ZZFeatureMap,
    n_local,
    z_feature_map,
    zz_feature_map,
)
from qiskit_aer import AerSimulator
from qiskit_algorithms.optimizers import Optimizer
from qiskit_ibm_runtime import Sampler
from qiskit_machine_learning.algorithms import QSVC, VQC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.optimizers import COBYLA, CRS, SPSA
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.utils import algorithm_globals
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from ucimlrepo import fetch_ucirepo
from utils import RANDOM_SEED, display_confussion_matrix, get_dataset, show_metrics

# Desarrolla un QSVM

En este notebook probamos diferentes configuraciones de QSVMx parapoder elegir la más óptima.

Al igual que siempre, fijamos la semilla de los números aleatorios para que los resultados sean reproducibles.

In [2]:
algorithm_globals.random_seed = RANDOM_SEED

## Preprocesado del dataset

Utilizamos una función `get_dataset` del módulo `utilis.py` que hemos definido. El funcionamiento de esta función es exactamente el mismo que el de la sección de preprocesado del notebook [practica final](practica_final.ipynb).

In [3]:
X_train, y_train, X_test, y_test, X_val, y_val = get_dataset()

## Probar distintos QSVM

A continuación probamos diferentes clasificadores, los cuales se diferencian en el `feature_map`.

In [ ]:
def train_QSVC(feature_map) -> QSVC:
    sampler = Sampler(AerSimulator())
    fidelity = ComputeUncompute(sampler=sampler)
    kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    qsvc = QSVC(quantum_kernel=kernel)
    return _fit(qsvc)


def _fit(svc: SVC) -> SVC:
    global X_train, Y_train, X_test, Y_test

    start = time()
    svc.fit(X_train, Y_train)
    total_time = time() - start

    if total_time < 10:
        print(f"Tiempo total de entrenamiento: {total_time * 1000:.2} ms")
    else:
        print(f"Tiempo total de entrenamiento: {timedelta(seconds=total_time)}")

    return svc


def evaluate_SVC(svc: SVC) -> None:
    global X_train, Y_train, X_test, Y_test

    train_score = svc.score(X_train, Y_train)
    test_score = svc.score(X_test, Y_test)
    y_pred_svc = svc.predict(X_test)
    svc_recall = recall_score(Y_test, y_pred_svc, pos_label=0)

    print(f"SVC en entrenamiento: {train_score:.5}")
    print(f"SVC en test (Accuracy): {test_score:.5}")
    print(f"SVC en test (Recall): {svc_recall:.5}")